# Ensemple CB recommender based on Sentence Transformers

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import torch
import numpy as np
import pandas as pd

import pytorch_common.util as pu

import logging
import data as dt

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.11.0'

## Evaluación

In [8]:
def load(name): return pd.read_json(f'../datasets/movie_{name}_embedding_bert.json')
def emb_rec(emb, name): return rc.DistanceMatrixRecommender(emb, f'{name}_embedding')

In [9]:
embeddings_by_name   = {name: load(name) for name in ['genres', 'overview', 'tags']}
recommenders_by_name = {name: emb_rec(emb, name) for name, emb in embeddings_by_name.items()}
emb_recommenders     = list(recommenders_by_name.values())

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

In [10]:
item_index = 10

In [11]:
recommender_group = rc.RecommenderGroup(emb_recommenders)

In [12]:
recommender_group.recommend(item_index).show()


Recommender: genres
Item


,id,title
10,8360,Shrek 2


Recommendations


,index,distance,id,title,genres
0,10,0.000000,8360,Shrek 2,Adventure Animation Children Comedy Musical Ro...
1,2860,0.017834,2081,"Little Mermaid, The",Animation Children Comedy Musical Romance
2,2739,0.019455,56152,Enchanted,Adventure Animation Children Comedy Fantasy Mu...
3,2858,0.027384,1907,Mulan,Adventure Animation Children Comedy Drama Musi...
4,7187,0.041949,88468,Alpha and Omega,Adventure Animation Children Comedy Romance



Recommender: overview
Item


...

,index,distance,id,title,overview
0,10,0.000000,8360,Shrek 2,"Shrek, Fiona and Donkey set off to Far, Far Aw..."
1,1916,0.259786,53121,Shrek the Third,The King of Far Far Away has died and Shrek an...
2,2954,0.333089,78637,Shrek Forever After,A bored and domesticated Shrek pacts with deal...
3,14344,0.439909,64249,Shrek the Halls,The Christmas tree isn't the only thing green ...
4,3,0.447712,4306,Shrek,It ain't easy bein' green -- especially if you...



Recommender: tags
Item


,id,title
10,8360,Shrek 2


Recommendations


,index,distance,id,title,tags
0,10,0.000000,8360,Shrek 2,(s)vcd 2020-2 relook 4 andrew adamson animatio...
1,2733,0.193913,45517,Cars,actually funny aftercreditsstinger animation a...
2,1916,0.223048,53121,Shrek the Third,adventure ambush animation assault bad sequel ...
3,3,0.226049,4306,Shrek,3 adventure almost favorite andrew adamson ani...
4,19,0.233767,78499,Toy Story 3,action adventure alive toys animation animação...


In [13]:
recommender = rc.EnsembleWeightedDistanceMatrixRecommender(
    emb_recommenders,
    weights = { 'tags': 0.35, 'overview': 0.3, 'genres': 0.35}
)

In [15]:
recommender.recommend(item_index, k=20).show()


Recommender: ['genres', 'overview', 'tags']
Item


,id,title
10,8360,Shrek 2


Recommendations


,distance,title,genres,overview,tags
0,0.000000,Shrek 2,Adventure Animation Children Comedy Musical Ro...,"Shrek, Fiona and Donkey set off to Far, Far Aw...",(s)vcd 2020-2 relook 4 andrew adamson animatio...
1,0.062748,Shrek the Third,Adventure Animation Children Comedy Fantasy,The King of Far Far Away has died and Shrek an...,adventure ambush animation assault bad sequel ...
2,0.076379,Shrek,Adventure Animation Children Comedy Fantasy Ro...,It ain't easy bein' green -- especially if you...,3 adventure almost favorite andrew adamson ani...
3,0.091236,Shrek Forever After,Adventure Animation Children Comedy Fantasy IMAX,A bored and domesticated Shrek pacts with deal...,3d action animation bad sequel computer animat...
4,0.098943,Aladdin,Adventure Animation Children Comedy Musical,Princess Jasmine grows tired of being forced t...,(s)vcd 2d animation action adventure aftercred...
5,0.101551,Enchanted,Adventure Animation Children Comedy Fantasy Mu...,The beautiful princess Giselle is banished by ...,========================= alan menken amy adam...
6,0.103153,Frozen,Adventure Animation Comedy Fantasy Musical Rom...,Young princess Anna of Arendelle dreams about ...,2014 angry feminism animation awful story bad ...
7,0.109291,Tangled,Animation Children Comedy Fantasy Musical Roma...,When the kingdom's most wanted-and most charmi...,3d adapted from:book alan menken animal sideki...
8,0.109842,"Little Mermaid, The",Animation Children Comedy Musical Romance,This colorful adventure tells the story of an ...,2d animation 55 movies every kid should see--e...
9,0.110018,Toy Story 2,Adventure Animation Children Comedy Fantasy,"Andy heads off to Cowboy Camp, leaving his toy...",2009 reissue in stereoscopic 3-d 3d abandonmen...
